# Angad Kalra
# SN: 1005134999


# Primary Questions
* Q1: For all active fulltime NYT movie critics, how often do they correctly predict box office hits? 
* Q2: Does movie review sentiment align with whether it was a critic's pick or not?
* Q3: What characterisitcs are common among movies that do really well? Among movies that do poorly? 

# TO DO:
* Get all data into dataframes
* Go through each Q and complete
* Write Report

# Data Collection

In [82]:
import numpy as np
import pandas as pd
import requests, json

# Sample data collection code

# API urls & keys
nyt_url = "http://api.nytimes.com/svc/movies/v2"
nyt_apikey = "72aaefb1f009451e986a0e446468f649"

tmdb_url = "https://api.themoviedb.org/3"
tmdb_apikey = "49200255c3dc5d6af15e04656ea5f7c4"

# List of critics
url = nyt_url + "/critics/full-time.json"
critics_res = requests.get(url, params={"api-key": nyt_apikey})
critics_res = json.loads(critics_res.text)
critics = [x["display_name"] for x in critics_res["results"]]

In [100]:
# Table of movies + financial outcome
url = nyt_url + "/reviews/search.json"
critics_list = []
movie_titles = []
box_office_total = []
box_office_hit = []

# Get critic pick movies for each critic + info and create dataframe.
for c in critics:
    resp = requests.get(url, params={"api-key": nyt_apikey, "critics-pick": "Y", "reviewer": c})
    if resp.status_code == 200:
        resp = json.loads(resp.text)
    else:
        continue
        
    results = resp["results"]
    
    for r in results:
        tmdb_resp = requests.get(tmdb_url + "/search/movie", 
                            params={"api_key": tmdb_apikey, "query": r["display_title"], 
                                    "primary_release_year": int(r["publication_date"][0:4]) })
        
        if tmdb_resp.status_code == 200:
            tmdb_resp = json.loads(tmdb_resp.text)

            if (tmdb_resp["total_results"] > 0):
                mid = tmdb_resp["results"][0]["id"]
                movie_info = requests.get(tmdb_url + "/movie/{}".format(mid), params={"api_key": tmdb_apikey})

                if movie_info.status_code == 200:
                    movie_info = json.loads(movie_info.text)
                    
                    critics_list.append(c)
                    movie_titles.append(movie_info["title"])
                    box_office_total.append(movie_info["revenue"])
                    box_office_hit.append(1) if movie_info["revenue"] >= 100000000 else box_office_hit.append(0)


0
0
79275328
2525738
64414761
445435700
282382
0
119520023
2341534
3162153
0
65046687
285930
1818075
10367161
0
0
140705322
0
501912068
13498185
6046104
1041955
162589
868429911
0
1163056
64207623
147988192
0
0
1346739107
56996304
0
0
6700
0
2000000
0
4611
0


In [104]:
df_dict = {"critic": pd.Series(critics_list, dtype=str), "movie_title": pd.Series(movie_titles, dtype=str), 
                        "box_office_total": pd.Series(box_office_total, dtype=np.int64), "box_office_hit": pd.Series(box_office_hit, dtype=np.bool)}
box_office_movies = pd.DataFrame(df_dict)

In [105]:
box_office_movies.head

<bound method NDFrame.head of             critic                    movie_title  box_office_total  \
0      A. O. Scott                Minding the Gap                 0   
1      A. O. Scott                   Custody Road                 0   
2      A. O. Scott                     Hereditary          79275328   
3      A. O. Scott                 First Reformed           2525738   
4      A. O. Scott                         Fences          64414761   
5      A. O. Scott                     La La Land         445435700   
6      A. O. Scott                 Things to Come            282382   
7      A. O. Scott             Two Trains Runnin'                 0   
8      A. O. Scott                         Allied         119520023   
9      A. O. Scott                           Elle           2341534   
10     A. O. Scott             The Eagle Huntress           3162153   
11     A. O. Scott                    Fire at Sea                 0   
12     A. O. Scott                      Moonlig

In [ ]:
# Table of movies + review sentiment


# Table of movies + characteristics + revenue

* I'm going to create a table of movies for each critic and also get the info on how much money the movies made in theatres. I will use the NYT and Box Office Mojo APIs for this info.
* For question #2, I'm going to scrape the NYT movie reviews from the url provided in the API response and create a dataset that has the movie info, the review, sentiment analysis, and whether it was a critic's pick. I will use NYT API for this info.
* For question #3, I'll create two datasets: one for movies that did well and one for movies that performed poorly. I will use NYT, OMDb, and Box Office Mojo APIs for this.

# EDA

* Q1: I'm going to create a list of picks for each critic and see how many of their picks performed well in box office. I will define a "box office hit" as a movie that made more than 100 million dollars domestic. 
* Q2: After scraping the reviews off the web, I will run sentiment analysis on the reviews using popular Python packages (i.e TextBlob) and classify them. I will then compare the results to whether or not the movie was a critic's pick. Plotting the data seems like a good idea right now because it will allow me to see if their is a somewhat sigmoid relationship between the two variables. My plot will be 2D, with the x-axis being the different sentiment results and the y-axis being binary (+1 for critic pick, 0 otherwise).
* Q3: For each table (movies that did well, movies that did poorly), I will perform something similar to a set intersection among their characteristics and see what they have in common. I will also remove any characteristics that appear in both tables. 

# Modelling

* Q1: Don't think I need any formal model here. Just going to look at accuracy for each critic.

* Q2: Can't think of a formal model for this. I think I will just plot sentiment vs critic pick and see whether it looks like a sigmoid function.

* Q3: I'm thinking of using PCA to find which "features" of movies are the most influential when it comes to determining revenue. 